# Arduino Nano BLE33 Sense - Firmware Upload

Upload the telescope sensor firmware to your Arduino Nano BLE33 Sense.

## Quick Start

1. **Double-tap the reset button** on the Arduino (LED will pulse green = bootloader mode)
2. **Run All Cells** (▶▶) in this notebook
3. Wait for "✅ Upload OK" and sensor data output

## Troubleshooting

| Problem | Solution |
|---------|----------|
| Upload timeout | Double-tap reset button, retry |
| "No device found" | Check USB cable, try different port |
| Sensors hang at init | Unplug USB, wait 5 sec, replug, double-tap reset |
| Permission denied | Run: `sudo usermod -a -G dialout $USER` (then logout/login) |

## Hardware

- **Board**: Arduino Nano 33 BLE Sense
- **Sensors**: LSM9DS1 (IMU), HTS221 (temp/humidity)
- **Serial**: 115200 baud, outputs tab-separated values at 10Hz

In [7]:
# Setup: imports and arduino-cli configuration
import subprocess
import serial
import serial.tools.list_ports
import time
import os
from pathlib import Path

# Add ~/bin to PATH for arduino-cli
home_bin = Path.home() / "bin"
if home_bin.exists():
    os.environ["PATH"] = f"{home_bin}:{os.environ.get('PATH', '')}"

# Verify arduino-cli is available
result = subprocess.run(["arduino-cli", "version"], capture_output=True, text=True)
print(f"✓ {result.stdout.strip()}")

# Board configuration
BOARD = "arduino:mbed_nano:nano33ble"
SKETCH = Path("../arduino/telescope_sensors")

# Auto-detect Arduino Nano 33 BLE port
PORT = None
for port in serial.tools.list_ports.comports():
    # Look for Arduino Nano 33 BLE (VID:PID = 2341:005A or 2341:805A for bootloader)
    if port.vid == 0x2341 and port.pid in (0x005A, 0x805A):
        PORT = port.device
        print(f"✓ Found Nano 33 BLE: {port.device}")
        break
    # Fallback: any Arduino on ACM port
    elif "Arduino" in (port.description or "") or (port.vid == 0x2341):
        PORT = port.device
        print(f"✓ Found Arduino: {port.device} ({port.description})")
        break

if not PORT:
    # Last resort: first ACM port
    acm_ports = [p for p in serial.tools.list_ports.comports() if "ACM" in p.device]
    if acm_ports:
        PORT = acm_ports[0].device
        print(f"⚠️ No Arduino detected, using first ACM port: {PORT}")
    else:
        print("❌ No serial port found! Is Arduino connected?")
        print("   Available ports:")
        for p in serial.tools.list_ports.comports():
            print(f"     {p.device}: {p.description}")

print(f"✓ Board: {BOARD}")
print(f"✓ Port: {PORT}")
print(f"✓ Sketch: {SKETCH.resolve()}")

✓ arduino-cli  Version: 1.4.0 Commit: b7000970f Date: 2025-12-09T15:55:00Z
✓ Found Nano 33 BLE: /dev/ttyACM0
✓ Board: arduino:mbed_nano:nano33ble
✓ Port: /dev/ttyACM0
✓ Sketch: /home/mark/src/telescope-mcp/arduino/telescope_sensors


In [8]:
# Compile and upload telescope_sensors firmware
# ⚠️ Double-tap reset button BEFORE running if upload fails!

print(f"📤 Compiling and uploading {SKETCH.name}...")
print(f"   Target: {BOARD} @ {PORT}\n")

try:
    result = subprocess.run([
        "arduino-cli", "compile", "--upload",
        "--fqbn", BOARD, "--port", PORT,
        str(SKETCH.resolve())
    ], capture_output=True, text=True, timeout=60)
except subprocess.TimeoutExpired:
    print("❌ Upload timeout!")
    print("   → Double-tap the reset button on Arduino")
    print("   → Wait for pulsing green LED (bootloader mode)")
    print("   → Re-run this cell")
else:
    if result.returncode != 0:
        print("❌ Upload failed!")
        print(result.stderr)
    else:
        print("✅ Upload OK!\n")
        print("Waiting for Arduino to restart...")
        time.sleep(2)
        
        # Read sensor output to verify it's working
        ser = serial.Serial(PORT, 115200, timeout=2)
        print("\n📊 Sensor output (10 seconds):")
        print("-" * 60)
        
        start = time.time()
        line_count = 0
        while time.time() - start < 10:
            line = ser.readline().decode(errors='ignore').strip()
            if line:
                # Show debug lines and first few data lines
                if line.startswith(("DEBUG:", "INFO:", "ERROR:", "telescope_sensors")):
                    print(f"  {line}")
                elif line_count < 5:  # Show first 5 data lines
                    print(f"  DATA: {line}")
                    line_count += 1
                elif line_count == 5:
                    print(f"  ... (sensor streaming at 10Hz)")
                    line_count += 1
        
        ser.close()
        print("-" * 60)
        print("\n✅ Firmware upload complete! Sensors are streaming.")
        print("   Use arduino-nano-ble33-sense.ipynb to read sensor data.")

📤 Compiling and uploading telescope_sensors...
   Target: arduino:mbed_nano:nano33ble @ /dev/ttyACM0

✅ Upload OK!

Waiting for Arduino to restart...

📊 Sensor output (10 seconds):
------------------------------------------------------------
  INFO: Initializing LSM9DS1 IMU...
  DEBUG: Calling IMU.begin()...
  DEBUG: IMU.begin() returned OK
  INFO: IMU warm-up...
  DATA: OK: IMU initialized
  INFO: Initializing HTS221 sensor...
  INFO: HTS warm-up...
  DATA: OK: HTS221 initialized
  DATA: OK: All sensors initialized successfully
  DEBUG: Init complete, starting loop
  DATA: 0.98	-0.01	-0.06	43.49	-24.65	2.09	27.51	42.07
  DATA: 0.98	-0.01	-0.06	43.48	-25.06	2.76	27.45	42.12
  ... (sensor streaming at 10Hz)
------------------------------------------------------------

✅ Firmware upload complete! Sensors are streaming.
   Use arduino-nano-ble33-sense.ipynb to read sensor data.
